In [8]:
from selenium import webdriver
import time
import pandas as pd

def login(driver:webdriver.Chrome, user_email:str, user_pw:str) -> None:
    driver.get('https://wehelp.tw/user')
    view_elements = driver.find_elements(by='css selector', value='div.view')
    if len(view_elements) > 0:
        return
    else:
        mail_button = driver.find_element(by='name', value='email')
        mail_button.send_keys(user_email)
        pw_button = driver.find_element(by='name', value='password')
        pw_button.send_keys(user_pw)
        login_button = driver.find_element(by='name', value='btn')
        login_button.click()
        time.sleep(2)
        return
    
def drive_to_reports(driver:webdriver.Chrome) -> None:
    report_url = 'https://wehelp.tw/bootcamp/learning-report'
    if driver.current_url != report_url:
        driver.get(report_url)
        time.sleep(2)
    
def roll_page(driver:webdriver.Chrome) -> None:
    drive_to_reports(driver)
    loadmore_button = driver.find_element(by='id', value='loadmore')
    loadmore_button.click()
    time.sleep(2)

def download_data(driver:webdriver.Chrome) -> pd.DataFrame:
    col_names = ['name-time', 'done', 'todo', 'memo', 'likes', 'comments']
    temp = {}
    js_str = """
    var nameTimeDivs = document.getElementsByClassName(arguments[0]);
    var texts = [];
    for (var i = 0; i < nameTimeDivs.length; i++) {
        texts.push(nameTimeDivs[i].textContent);
    }
    return texts;
    """
    for col in col_names:
        name_time_texts = driver.execute_script(js_str, col) # col 將傳入 js_str 的 arguments[0]
        temp[col] = name_time_texts
    df = pd.DataFrame(temp)
    df['name'] = [i[12:] for i in df["name-time"]]
    return df

def main(driver:webdriver.Chrome, user_email:str, user_pw:str, pages:int) -> pd.DataFrame:
    login(driver, user_email, user_pw)
    for _ in range(pages):
        try:
            print(f'Downloading page {_}.')
            roll_page(driver)
        except:
            print(f'Fail to download page {_}.')
            break
    return download_data(driver)

In [ ]:
# driver = webdriver.Chrome(r"chromedriver.exe") driver放到.py同路徑目錄，不需要設定路徑
driver = webdriver.Chrome()
df = main(driver, user_email, user_pw, 10)

In [12]:
df

,name-time,done,todo,memo,likes,comments,name
0,07-07 21:34 廖致皓,1. 複習Javascript DOM tree2. 與同學開Meeting討論第一週作業內...,1. 複習Python語法＆Flask(今天時間不夠沒複習到)2. 了解OAuth流程3. ...,今天很高興能和四位同學一起開meeting互相討論第一週作業的完成方式，學到幾種切版的思考方...,Be the first to like thisLike,,廖致皓
1,07-07 21:26 賴亭如,1. 完成第一周延伸作業。2. 學習 JavaScript。,1. JavaScript 筆記整理、實作練習。2. Python 學習。,1. 第一周的作業是用很基礎的方式完成， 暫時放棄研究華麗的功能，先求有 ~2. Ja...,Be the first to like thisLike,,賴亭如
2,07-07 20:44 吳伯韋,1.Python 裝飾器工廠練習。2.Flex、Grid切版練習。,1.Python Flask學習。2.學習 JS的可處理事件。,1.看到漢堡都會想到Python的裝飾器。2.感謝同學，現在看到蛋餅就想到grid。,Be the first to like thisLike,,吳伯韋
3,07-07 19:56 許秀瑜,1. 研究如何修正老師提出的地方2. 完成第一週作業3. 助教及同學提供的學習資源整理完畢（...,1. 把第一週作業從頭再寫過2. 要更熟悉 CSS 和 HTML 的語法3. 切版作業速度加速,身為程式小白現在只求有、不求好，但覺得自己寫出來的程式真的是爛爆，希望透過反覆練習能讓自己更...,Be the first to like thisLike,,許秀瑜
4,07-07 19:16 游珮姿,1. 利用Grid Garden練習CSS Grid概念2. JavaScript影片3....,1. JavaScript影片＆練習2. Python影片,1. 透過寫文章的方式review，把學到的東西理解後再寫成其他人看得懂內容，更能加深印象2...,Be the first to like thisLike,,游珮姿
...,...,...,...,...,...,...,...
215,02-02 00:09 劉彥成,1. 拖曳事件改以使用者觸發事件後註冊2. 將SVG畫布本身的拖曳事件改以下方工具列轉換3....,1. 繼續往節點邁進2. 繼續產生工具列及基礎功能,1. 拖曳事件清除大部分Bug，但還是有小蟲在搔癢中，只能先把節點和線段生出來，之後再來優化...,8 like thisLike,,劉彥成
216,02-01 23:09 陳思婷,1. 各種與firebase互動，能夠點擊區域拿到資料並同時運用資料串API拿到及時熱門景點...,1. 參加馬丁與峰林助教的Office Hour2. 把資料顯示到頁面上，並進一步拿到熱門景...,1. 第三階段真的很容易碰壁低落，雖然沒有用到什麼嶄新厲害的技術，但我想把這些基礎的用熟練也...,8 like thisLike,,陳思婷
217,02-01 07:47 黃柏瑄,1. 為什麼我會這個時間寫報告呢 因為剛剛載夫人去機場他要去泰國可惡的女人2. 花太多時間在...,1. 這禮拜至少要把資料庫跟所有頁面都搞出來之後終於可以排版了現在有夠醜也不夠直覺3. 還要...,1. 真的花太多時間在資料庫跟storage了2. Firebase Database 跟 ...,13 like thisLike,,黃柏瑄
218,01-31 21:38 侯晨曦,1. 完成註冊功能，包含前後端 regex 驗證及提示，使用者及信箱是否重複及提示,1. 登入功能2. 研究 Django Session,Discord 的細節真的很多，沒想到光模仿一個註冊功能就寫了 200 行註冊成功的畫面沒截...,6 like thisLike,,侯晨曦


In [ ]:
# like 點開之後會有一個選單，裡面顯示那些人按讚
# comment 點擊之後會有所有人的回應，如果沒有留言，就不會有可以點擊的comment
# timestamp 載入網頁之後會直接捨棄年份，但是javascript應該可以取到timestamp
# df.to_csv('wehelp_reports.csv', encoding='utf-8-sig')